# Analisis del mercado de criptomonedas - creacion de datasets

In [1]:
import pandas as pd
import numpy as np
import os
import requests
from datetime import datetime, timedelta
from pycoingecko import CoinGeckoAPI
from binance import Client

In [2]:
# Instancia la API de coingecko
cg = CoinGeckoAPI()

# Bloque para definir funciones

In [3]:
# Retorna el top 10 de monedas con mayor capitalizacion de mercado en USD
def get_top10_marketcap():
    banned_coins = ['staked-ether', 'usd-coin', 'tether', 'solana', 'matic-network', 'polkadot', 'shiba-inu', 'wrapped-bitcoin', 'uniswap', 'the-open-network', 'avalanche-2', 'dai', 'dogecoin', 'bitcoin-cash', 'leo-token', 'binance-usd', 'true-usd', 'okb', 'chainlink', 'monero', 'cosmos', 'hedera-hashgraph', 'filecoin', 'internet-computer', 'lido-dao', 'aptos', 'mantle', 'crypto-com-chain', 'arbitrum', 'quant-network']
    top10_coins = []
    coins_markets_list = cg.get_coins_markets(vs_currency='usd')
    for i in coins_markets_list:
        if len(top10_coins) < 10:
            for key, x in i.items():
                if key == 'id' and x not in banned_coins:
                    top10_coins.append(x)
    return top10_coins

# Retorna un dataframe con los datos historicos de una moneda vs usd, ej: BTC/USD
def get_coin_data(coin):
    # Solicita datos historicos de las criptomonedas
    coin_data = cg.get_coin_market_chart_by_id(id=coin,vs_currency='usd',days='max')
    # Genera una lista de timestamps
    timestamp_list = []
    for key, value in coin_data.items():
        if key == 'prices':
            for x in value:
                timestamp_list.append(x[0])
    # Elimina los timestamps del diccionario que contiene los datos historicos
    for key, value in coin_data.items():
        for z in value:
            z.pop(0)
    # Crea el dataframe 
    df = pd.DataFrame(coin_data)
    df = df.applymap(lambda x: x[0])
    df['time'] = pd.Series(timestamp_list)
    df['time'] = df['time'].apply(lambda x: int(x / 1000))
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df[['time', 'prices', 'market_caps', 'total_volumes']]
    return df

# Obtiene un diccionario con los dataframes de las 10 monedas seleccionadas
def get_top10_dataframes():
    dataframes = {}
    top10_coins = get_top10_marketcap()
    for name in top10_coins:
        dataframes[name] = get_coin_data(name)
    return dataframes

# Iguala la longitud de todos los dataframes
def get_equal_lens(dataframes):
    # Genera un diccionario con la longitud de cada dataframe
    lens_dict = {}
    min_len = ()
    for key, value in dataframes.items():
        lens_dict[key] = len(value)
    # Encuentra el dataframe con menor longitud
    for key, value in lens_dict.items():
        if len(min_len) == 0:
            min_len = (key, value)
        else:
            if value < min_len[1]:
                min_len = (key, value)
    # Filtra por tiempo para igualar la longitud de todos los dataframes
    for key, value in dataframes.items():
        dataframes[key] = value.loc[value['time'] >= dataframes[min_len[0]]['time'].min()]
        dataframes[key].reset_index(inplace=True)
        dataframes[key] = dataframes[key].drop(columns=['index'])
        if dataframes[key]['time'].iloc[-1].hour != 0:
            dataframes[key] = dataframes[key].iloc[:-1]

# Obtiene un dataframe con datos OHLCV historicos de Binance Spot para la moneda seleccionada
def get_binance_ohlcv(symbol, interval, start_time, end_time):
    AK = 'HBeG4ssHjabdHao3blmB4ZjXx2EYUWAhVdyQ541fllwe9wHIczC8ah5qqa53n7Nw'
    SK = 'J1ibFsLIHxHAaOz9xQLJfawKA7GTFkjfljwjIKOPK2CK4UdCRuCPmAoiQlTF75E7'
    client = Client(AK, SK)
    candles = client.get_historical_klines(symbol, interval, str(start_time), str(end_time))
    df = pd.DataFrame(candles, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Empty'])
    df.drop(columns=['Empty', 'Quote asset volume', 'Taker buy quote asset volume'], inplace=True)
    target_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Taker buy base asset volume']
    for i in target_cols:
        df[i] = df[i].astype(float).apply(lambda x: round(x, 2))
    df['Open time'] = df['Open time'].apply(lambda x: int(x / 1000))
    df['Open time'] = pd.to_datetime(df['Open time'], unit='s')
    df['Close time'] = df['Close time'].apply(lambda x: int(x/ 1000))
    df['Close time'] = pd.to_datetime(df['Close time'], unit='s')
    return df

# Obtiene un diccionario con los dataframes de las 10 monedas seleccionadas en tempoiralidad de 1 dia
def get_top10_binance_1D():
    current_dt = datetime.now()
    yesterday_dt = current_dt - timedelta(days=1)
    dfs = {}
    pairs_list = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'ADAUSDT', 'TRXUSDT', 'LTCUSDT', 'XLMUSDT', 'ETCUSDT', 'VETUSDT']
    for name in pairs_list:
        dfs[name] = get_binance_ohlcv(symbol=name, interval=Client.KLINE_INTERVAL_1DAY, start_time='2017-11-09', end_time=str(yesterday_dt))
    return dfs

In [4]:
top10_coins = get_top10_marketcap()
top10_coins

['bitcoin',
 'ethereum',
 'binancecoin',
 'ripple',
 'cardano',
 'tron',
 'litecoin',
 'stellar',
 'ethereum-classic',
 'vechain']

In [5]:
dataframes = get_top10_dataframes()
get_equal_lens(dataframes)
dataframes

{'bitcoin':            time        prices   market_caps  total_volumes
 0    2018-07-27   7929.079244  1.361666e+11   3.429994e+09
 1    2018-07-28   8145.333168  1.398974e+11   3.624913e+09
 2    2018-07-29   8192.135702  1.407162e+11   3.659115e+09
 3    2018-07-30   8197.593576  1.408274e+11   2.598593e+09
 4    2018-07-31   8185.921138  1.406408e+11   3.497813e+09
 ...         ...           ...           ...            ...
 1841 2023-08-11  29423.818916  5.724142e+11   8.106254e+09
 1842 2023-08-12  29396.847971  5.719766e+11   4.651950e+09
 1843 2023-08-13  29412.142275  5.722578e+11   3.994298e+09
 1844 2023-08-14  29284.969714  5.696871e+11   4.755220e+09
 1845 2023-08-15  29400.586804  5.722427e+11   1.240836e+10
 
 [1846 rows x 4 columns],
 'ethereum':            time       prices   market_caps  total_volumes
 0    2018-07-27   460.869334  4.652016e+10   1.425779e+09
 1    2018-07-28   467.957927  4.724524e+10   1.318960e+09
 2    2018-07-29   465.899155  4.704692e+10   1.5896

In [6]:
dfs = get_top10_binance_1D()

# Bloques de union de dataframes

In [7]:
# Define un diccionario con nuevas claves
new_keys_mapping = {'BTCUSDT': 'bitcoin', 'ETHUSDT': 'ethereum', 'BNBUSDT': 'binancecoin', 'XRPUSDT': 'ripple', 'ADAUSDT': 'cardano', 'TRXUSDT': 'tron', 'LTCUSDT': 'litecoin', 'XLMUSDT': 'stellar', 'ETCUSDT': 'ethereum-classic', 'VETUSDT': 'vechain'}

# Modifica el diccionario original con las claves cambiadas
for old_key, new_key in new_keys_mapping.items():
    if old_key in dfs:
        dfs[new_key] = dfs.pop(old_key)

In [8]:
# Hace un merge entre los dataframes provenientes de coingecko y de binance
for key, value in dataframes.items():
    dataframes[key].rename(columns={'time': 'Open time'}, inplace=True)
    dataframes[key] = dataframes[key].merge(dfs[key], on='Open time', how='inner')

In [9]:
# Filtra y selecciona las columnas de interes
for key, value in dataframes.items():
    dataframes[key] = value[['Open time', 'Close', 'total_volumes', 'market_caps']]

In [10]:
import os
# Ruta a la carpeta que contiene los archivos CSV con data adicional de binance
folder_path = 'Binance top10 datasets'

# Obtiene la lista de archivos en la carpeta
file_list = os.listdir(folder_path)

# Filtra solo los archivos CSV
csv_files = [file for file in file_list if file.endswith('.csv')]

# Itera a través de los archivos CSV
sorted_top10_coins = ['cardano', 'binancecoin', 'bitcoin', 'ethereum-classic', 'ethereum', 'litecoin', 'tron', 'vechain', 'stellar', 'ripple']
for idx, csv_file in enumerate(csv_files):
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    df.drop(columns=['Open', 'High', 'Low', 'Close'], inplace=True)
    df['Open time'] = pd.to_datetime(df['Open time'])
    dataframes[sorted_top10_coins[idx]] = dataframes[sorted_top10_coins[idx]].merge(df, on='Open time', how='inner')

In [11]:
# Crea la columna binaria de clasificacion direccional y se transforma los valores numericos continuos a diferencias porcentuales
target_columns = ['Close', 'total_volumes', 'market_caps', 'POC']
for key, value in dataframes.items():
    dataframes[key]['Directional_bias'] = np.where(value['Close'] < value['POC'], 1, 0)
    for i in target_columns:
        dataframes[key][i] = dataframes[key][i].pct_change() * 100
    dataframes[key]['Next_close'] = dataframes[key]['Close'].shift(-1)
    dataframes[key].dropna(inplace=True)

In [12]:
# Crea el retorno de la estrategia
for key, value in dataframes.items():
    condition_1 = (dataframes[key]['Directional_bias'] == 1) & (dataframes[key]['Next_close'] > 1)
    condition_2 = (dataframes[key]['Directional_bias'] == 0) & (dataframes[key]['Next_close'] < 1)
    dataframes[key]['Strategy_returns'] = np.where(condition_1 | condition_2, dataframes[key]['Next_close'] * (-1), dataframes[key]['Next_close'])

In [13]:
# Crea archivos csv con los dataframes finales
for key, value in dataframes.items():
    dataframes[key].to_csv(f'Final dataframes returns\{key}.csv', header=True, index=False)